# Projeto 1 - Ciência dos Dados

Nome: Mia Valentin Machado

Nome: Matheus Castellucci

Nome: Pedro Henrique

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
#import emoji

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Matheus Castellucci\Desktop\Insper\Semestre 2\CieDados\Projeto 1\Projeto-1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
PS5 = 'PS5 - Final.xlsx'

In [5]:
train = pd.read_excel(PS5)
train.head(5)

,Tweet,Classificação de Relevância
0,um ps5 eh pra sempre https://t.co/ow1aobrsmm,2
1,"vi uma galerinha falando: ""ain que maravilha w...",0
2,"@jisegirl os dois, você no sofa, o ps5 no rack...",0
3,vender meu pulmão pra comprar um ps5 https://t...,2
4,@midiasonysta @playstation_br endosso o coro.....,1


In [6]:
test = pd.read_excel(PS5, sheet_name = 'Teste')
test.head(5)

,Tweet,Classificação de Relevância
0,"@wagnerthomazoni diria que é bom até, mais tem...",2
1,@d_ultra_ @esportzy @vplportugalps eu já que s...,1
2,@guisouzadias branco com ps5 em casa adora def...,0
3,@oblivionhell jogando no meu ps5,2
4,"queria pegar um console mais barato, mas exclu...",3


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é o PS5, um console mais recente da Sony, uma empresa predominante no mercado de eletrônicos. Diante disto, consideramos como relevantes tweets com teor critico a cerca do console e suas particuliaridades, classificando então as opniões sobre seus jogos e manisfestações sobre o desejo de possuir um PS5, como tweets irrelevantes ou neutros dependendo do teor do tweet que os um possível comprador 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [7]:
def limpa(tweet):
#Função responsável por retirar os caractéres especiais (! - . : ? ; \n) e os links

    tweet_sem_carac= re.sub(re.compile('[!-.:?;\n]'), '', tweet)
    tweet_limpo = re.sub(re.compile(r'(http\S*\s)'), '', tweet_sem_carac)
    #substitui os caracteres especiais e links por um espaço

    return tweet_limpo

def espacamento(tweet):
    text_list = ''
    #Variável para receber os tweets com o espaçamento correto
    
    for palavra in tweet:
        if palavra in emoji.UNICODE_EMOJI:
            text_list += f' {palavra} '
        else:
            text_list += f'{palavra}'
    #Verifica se cada palavra do tweet é um emoji ou não, cao sim, o programa aplicará um espaçamento antes e depois dele.
    
    tweet_corrigido = re.sub(re.compile(r'\s+'), ' ', text_list) 
    #Verifica se existe casos de espaçamento duplo e então transforma-o em um espaçamento simples
    
    return tweet_corrigido

In [8]:
classes = [('mt_ir', 0), ('ir', 1), ('nt', 2), ('rl', 3)] 
#lista dos nomes e dos respectivos numeros das classificações usadas

string = {}                                         
#Dicionario que separará os tweets por classificação

for classe, n in classes:

    classifier = train.loc[train['Classificação de Relevância'] == n, :]      
    #Seleciona os tweets de uma classe
    
    string[classe] = ''
    #cria uma string para armazenar os tweets ´por classe
    
    for palavra in classifier['Tweet']:
        string[classe] += ' ' + palavra
    #Cria o novo tweet, agora limpo
    
    string[classe] = limpa(string[classe].lower()) #faz a limpeza da string
string['ir'][:1000]

' @midiasonysta @playstation_br endosso o coro blader é um dos meus criadores de conteúdo preferido no youtubesempre traz debates e opiniões pertinentes aqui no twittere é sensacional@bladerkoyotte merece demais um ps5pense nisso sonya o ps5 chegou @ataylindaa mas c viu que vai sair godofwar p ps5  @dekostrife será enorme com certeza o cory está neste projeto indica o quanto a sony aposta nele sendo só ps5 ainda o hype sobe mais ainda vendi meio ethereum pra comprar um ps5 e depois desisti da ideia recomprei mais alto o ethereum boa noite já pesquisou xbox ps5 e nft pra checar se vc não ta seguindo uma página que foi vendida/hackeada hj @nerdesports021 clubes pagam as ps5  consola foi para venda e mundo entrou em pandemia qual a vantagem de comprar uma ps5 atualmente  já há exclusivos ps5  @_cgoulxrt nosso filho vai desenhar no ps5 sim @lord_edipo @jdiasssxl @nerdboomer isso é diferente a pirataria permitia você refazer a base e colocar no cd agora no ps4 e ps5 o trabalho é gigantescon

In [9]:
t = 1           #print da quantidade de palavras em cada classe na base de treino
nomes = {'mt_ir': 'Muito Irrelevante', 'ir': 'Irrelevante', 'nt': 'Neutro', 'rl': 'Relevante'}
for classe, n in classes:
    print(t)
    t += len(pd.Series(string[classe].split()).value_counts())
    
for classe, n in classes:
    print(f'palavras na classe {classe}:',len(pd.Series(string[classe].split()).value_counts()), f'({(len(pd.Series(string[classe].split()).value_counts())/t)*100:.3g}%)')

1
1195
2056
2857
palavras na classe mt_ir: 1194 (30.6%)
palavras na classe ir: 861 (22.1%)
palavras na classe nt: 801 (20.5%)
palavras na classe rl: 1047 (26.8%)


In [10]:

nomes = {'mt_ir': 'Muito Irrelevante', 'ir': 'Irrelevante', 'nt': 'Neutro', 'rl': 'Relevante'} 
#nomes para usar print de forma mais elegante

In [11]:
def cassificador(tweet):
    tweet_limpo = limpa(tweet)
    tweet_correto = espacamento(tweet_limpo)
    tweet_palavra = tweet_clean.split()

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**